In [ ]:
#Task 1: Khai báo môi trường và các thư viện cần thiết; Cho biết các thư viện đó ứng dụng vào việc gì
import numpy as np
import random
import matplotlib.pyplot as plt     #Thư viện này để vẽ đồ thị
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping     #thư viện này để gọi hàm call back trong quá trình training
from tensorflow.keras.preprocessing.image import ImageDataGenerator       #thư viện này để gọi hàm tăng cường dữ liệu
from tensorflow.keras.preprocessing import image                          #dùng load image
from tensorflow.keras.models import Sequential                            #dùng xây dựng CNN tuần tự
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,
BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D                #nhập các lớp cho CNN
from tensorflow.keras.optimizers import Adam                              #import trình tối ưu hóa

%matplotlib inline

In [ ]:
#Task 2: định dạng hình ảnh, nạp dữ liệu train
im_shape = (250,250)    #xác định kích thước hình ảnh
seed = 10               #số trạng thái ngẫu nhiên sẽ được giữ nguyên qua các lần epoch
BATCH_SIZE = 20         #số ảnh được thực hiện trong 1 bước của epoch

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`    #Xóa class inynbcheckpoint có sẵn trong jupiter notebook dùng để ghi nhớ dữ liệu

In [ ]:
#Tạo tập generator, dùng để huấn luyện dữ liệu
data_generator = ImageDataGenerator(
        validation_split=0.2,         # Tỉ lệ phần trăm dữ liệu được sử dụng cho validation set (tách từ dữ liệu huấn luyện)
        rotation_range=20,            # Góc quay ảnh (degrees)
        width_shift_range=0.2,        # Phạm vi dịch chuyển theo chiều ngang (tỷ lệ của bức ảnh)
        height_shift_range=0.2,       # Phạm vi dịch chuyển theo chiều dọc (tỷ lệ của bức ảnh)
        rescale=1./255,               # Tỉ lệ giảm độ sáng theo pixel
        shear_range=0.2,              # Góc cắt (shear) ảnh
        zoom_range=0.2,               # Tỉ lệ thu phóng ngẫu nhiên
        horizontal_flip=True,         # Lật ngang ảnh theo chiều ngang
        brightness_range=[0.8, 1.2],  # Phạm vi độ sáng của ảnh
        channel_shift_range=0.2,      # Phạm vi dịch chuyển kênh màu (RGB) theo chiều ngang
        vertical_flip=True,           # Lật dọc ảnh
        fill_mode='nearest')          # trong lúc biến đổi hình ảnh, sẽ xuất hiện các pixel trống và chúng được điền theo pixel gần nhất

val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)   #tạo tập valid_gen bằng cách lấy 20% số ảnh của tập train

In [ ]:
  #Task này có thay đổi ở tập test, sử dụng tập validation để tạo bảng confusion matrix
  train_generator = data_generator.flow_from_directory('/content/train', target_size=im_shape, shuffle=True, seed=seed,class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
  #tập train_generator có tác dụng tạo ra dữ liệu hình ảnh cho các batch, ở đây sử dụng size(250,250), xóa trộn ảnh để các batch luôn được ngẫu nhiên, class_mode categorical có sẽ biểu diễn label dưới dạng nhị phân theo số lớp, ví dụ như này: "Cat": [1, 0, 0], với mục đích (subset) là training
  validation_generator = val_data_generator.flow_from_directory('/content/train', target_size=im_shape, shuffle=False, seed=seed,class_mode='categorical', batch_size=BATCH_SIZE, subset="validation") #giống tập train

  nb_train_samples = train_generator.samples        # lưu ảnh từ tập vào biến để sử dụng
  nb_validation_samples = validation_generator.samples

  classes = list(train_generator.class_indices.keys())   # kiểm tra và in ra tên của các lớp trong tập huấn luyện
  print('Classes: '+str(classes))
  num_classes  = len(classes)

In [ ]:
#kiểm tra các ảnh được tạo ra và biến đổi từ tập generator
plt.figure(figsize=(12, 12))

for i in range(16):
    plt.subplot(4, 4, i+1)
    batch = train_generator.next()[0] * 255
    image = batch[0].astype('uint8')
    # Định dạng khác nhau
    if i % 2 == 0:
        plt.imshow(image, cmap='viridis')  # Định dạng cmap
    else:
        plt.imshow(image, cmap='gray')
    plt.title(f'Image {i+1}')
    # Ẩn các trục
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
model = Sequential()                                          #tạo mô hình tuần tự, dữ liệu lớp này sẽ chuyển tới lớp tiếp theo
model.add(Conv2D(20, kernel_size=(3, 3),                      #tạo lớp tích chập với 20 nơ ron, kernel là 1 ma trận dùng để học các đặc trưng không gian như cạnh góc, pattern cục bộ của dữ liệu, act = relu để học nhiều đặc trưng phi tuyến tính(là học mấy cái biểu diễn phức tạp), kích thước hình ảnh 250x250 và nhận kênh 3 màu RGB
                 activation='relu',
                 input_shape=(250,250,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))                     #giảm độ phức tạp, tham số cần học và kích thước của đầu ra từ lớp conv2d trước đó
model.add(Conv2D(40, kernel_size=(3,3), activation='relu'))   #thêm 1 lớp conv2d nữa để mô hình học sâu hơn, ví dụ lớp trc đó học được viền và cạnh góc gì đó thì giờ nó học sâu hơn là texture, pattern hoặc đối tượng cụ thể
model.add(Flatten())                                          #chuyển các đầu ra 2d hoặc 3d thành vector 1 chiều có chứa các đặc trưng để đưa vào lớp dense (dense là fully connected nên chỉ có 1 chiều thôi)
model.add(Dense(100, activation='relu'))                      #lớp này kết nối các nơ ron từ các lớp trước đó lại với nhau, nghĩa là mỗi nơ ron trong dense sẽ được kết nối 1 trong 100 nơ ron có cùng trọng số trước đó
model.add(Dropout(0.2))                                       #cắt bớt đặc trưng, giúp mô hình không quá phụ thuốc vào các đặc trưng mà còn có thể phân loại đa dạng hình ảnh hơn
model.add(Dense(num_classes, activation='softmax'))           #lớp kết nối cuối cùng với act = softmax để tạo ra bài toán phân loại, ví dụ trong này 3 lớp thì có 3 xác suất, cái xác suất nào lớn nhất thì chọn làm dự đoán prediction cuối cùng
model.summary()                                               #hiện bảng tóm tắt như ở dưới

model.compile(loss='categorical_crossentropy',                #khai báo hàm mất mát có sẵn trong thư viện của Keras
              optimizer=Adam(),                               #trình tối ưu hóa Adam
              metrics=['accuracy'])                           #chọn độ đo cho dự đoán, ở đây là tỉ lệ chính xác (độ đo là các phép tính toán như f1, recall, precision đồ đó)

In [ ]:
epochs = 15
# Định nghĩa các callbacks
checkpoint_callback = ModelCheckpoint(filepath='model.h5', monitor='val_loss', save_best_only=True, verbose=1)    #lưu trạng thái vào 1 tệp mỗi lần epoch được gọi, trong đó monitor theo dõi metrics val_loss, nếu như val_loss giảm so với lần epoch trước thì lưu, verbose để hiện thông báo là xong rồi thôi
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2, verbose=1)                                #nếu val_loss sau 2 epoch mà không giảm được thì hàm này được gọi để dừng train

# Gộp các callbacks vào một list
callbacks_list = [checkpoint_callback, early_stopping_callback]                                                   #dùng 2 callback này để tránh overfitting và tiết kiệm thời gian train

#Training
history = model.fit(                                                                                              #bắt đầu huần luyện
        train_generator,
        steps_per_epoch=nb_train_samples // BATCH_SIZE,                                                           #số bước trong 1 epoch = số ảnh / số ảnh trong 1 lô (batch)
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data=validation_generator,
        verbose = 1,
        validation_steps=nb_validation_samples // BATCH_SIZE)

In [ ]:
history_dict = history.history
#vẽ đồ thị loss theo từng epoch
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs_x = range(1, len(loss_values) + 1)
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.plot(epochs_x, loss_values, 'bo', label='Training loss')
plt.plot(epochs_x, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation Loss and Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')

#vẽ đồ thị acc theo từng epoch
plt.legend()
plt.subplot(2,1,2)
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs_x, acc_values, 'bo', label='Training acc')
plt.plot(epochs_x, val_acc_values, 'b', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model.h5')

In [ ]:
import itertools #

#Plot the confusion matrix. Set Normalize = True/False
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(model, data_generator, classes):
    # Dự đoán nhãn cho tập dữ liệu
    predictions = model.predict_generator(data_generator)
    predicted_labels = np.argmax(predictions, axis=1)

    # Lấy nhãn thực tế từ tập dữ liệu
    true_labels = data_generator.classes

    # Tính confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Hiển thị thông tin và vẽ confusion matrix
    print("Confusion Matrix:")
    print(cm)

    # Vẽ confusion matrix bằng cách sử dụng hàm plot_confusion_matrix
    plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix')

    # Classification Report
    print('\nClassification Report:')
    print(classification_report(true_labels, predicted_labels, target_names=classes))

# Sử dụng hàm evaluate_model
evaluate_model(model, validation_generator, classes)

In [ ]:
#Vừa xuất file đúng với tên ban đầu, vừa xuất ra màn hình với predicted label và được resize
from tensorflow.keras.preprocessing import image
import numpy as np
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục chứa ảnh test
test_dir = '/content/test'  # Điều chỉnh đường dẫn tới thư mục chứa ảnh test bên ngoài

# Tạo một list chứa đường dẫn đầy đủ đến từng ảnh trong thư mục test
test_images = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]

# Kích thước mới cho việc hiển thị
display_size = (250, 250)

# Khởi tạo một thư mục để lưu các ảnh dự đoán
output_dir = '/content/o5'  # Điều chỉnh đường dẫn đến thư mục output
os.makedirs(output_dir, exist_ok=True)

# Duyệt qua từng ảnh trong tập test
for i in range(0, len(test_images), 4):  # Process images in batches of 4
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))

    for j, img_path in enumerate(test_images[i:i+4]):
        # Đọc và resize ảnh về kích thước phù hợp với mô hình
        img = image.load_img(img_path, target_size=display_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
        # Dự đoán nhãn cho ảnh sử dụng model đã được train và lưu vào model.hd5 và gán thành biến model
        predictions = model.predict(img_array)
        predicted_label = np.argmax(predictions, axis=1)
        # Lấy tên nhãn dự đoán từ class_indices
        predicted_class = [k for k, v in train_generator.class_indices.items() if v == predicted_label[0]][0]
        # Tạo thư mục con cho mỗi nhãn nếu chưa tồn tại
        label_dir = os.path.join(output_dir, predicted_class)
        os.makedirs(label_dir, exist_ok=True)
        # Copy ảnh vào thư mục của nhãn
        shutil.copy(img_path, label_dir)

        # Hiển thị ảnh cùng với thông tin nhãn dự đoán
        img = Image.open(img_path)
        img_array = np.array(img.resize(display_size)) / 255.0  # Resize ảnh và chuẩn hóa giá trị pixel
        axes[j // 2, j % 2].imshow(img_array)
        axes[j // 2, j % 2].set_title(f"Predicted Label: {predicted_class}")

    plt.show()
